In [1]:
import cv2
import time
import numpy as np
import threading
from deepface import DeepFace
from tensorflow.keras.models import load_model
from keras.models import load_model

In [2]:
# Eğittiğiniz modeli yükleyin
model = load_model('model_dropout.h5')

In [ ]:
# Global değişkenler
recognized_name = ""
name_timestamp = 0
name_display_duration = 3  # saniye

emotion_label_text = ""
emotion_timestamp = 0
emotion_display_duration = 1.5  # saniye

def run_deepface(frame):
    global recognized_name, name_timestamp
    try:
        result = DeepFace.find(img_path=frame, db_path="dataset", enforce_detection=False)
        if len(result[0]) > 0:
            identity_path = result[0].iloc[0]["identity"]
            recognized_name = identity_path.split("\\")[-2]
        else:
            recognized_name = "Kisi bulunamadi"
        name_timestamp = time.time()
    except Exception as e:
        recognized_name = "Yuz algilanamadi"
        name_timestamp = time.time()
        print("Hata:", e)

# Kamera başlat
cap = cv2.VideoCapture(0)

fps_limit = 5
interval = 1.0 / fps_limit
last_time = time.time()

deepface_interval = 2  # saniyede bir çalışsın
last_deepface_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Duygu tahmini
    if time.time() - last_time >= interval:
        last_time = time.time()
        resized_frame = cv2.resize(frame, (48, 48))
        img_array = np.expand_dims(resized_frame, axis=0) / 255.0
        prediction = model.predict(img_array)
        prob_happy = prediction[0][0] * 100
        prob_sad = (1 - prediction[0][0]) * 100
        emotion_label = "Happy" if prediction[0][0] > 0.5 else "Sad"
        emotion_label_text = f"{emotion_label}: {max(prob_happy, prob_sad):.2f}%"
        emotion_timestamp = time.time()

    # Duygu metnini göster (örneğin 1.5 saniye boyunca)
    if time.time() - emotion_timestamp <= emotion_display_duration:
        cv2.putText(frame, emotion_label_text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 200, 0), 2)

    # DeepFace'i ayrı thread ile çalıştır
    if time.time() - last_deepface_time >= deepface_interval:
        last_deepface_time = time.time()
        threading.Thread(target=run_deepface, args=(frame.copy(),)).start()

    # Tanınan isim varsa göster
    if time.time() - name_timestamp <= name_display_duration:
        cv2.putText(frame, f"Name: {recognized_name}", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Kamera", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
25-05-07 19:32:36 - Searching [[[147 152 151]
  [147 152 151]
  [147 152 151]
  ...
  [122 129 132]
  [122 129 132]
  [122 129 132]]

 [[147 152 151]
  [147 152 151]
  [147 152 151]
  ...
  [122 129 132]
  [122 129 132]
  [122 129 132]]

 [[148 153 152]
  [148 153 152]
  [148 153 152]
  ...
  [122 129 132]
  [122 129 132]
  [122 129 132]]

 ...

 [[156 151 143]
  [156 150 144]
  [157 151 145]
  ...
  [ 92  97  96]
  [ 93  98  97]
  [ 94  99  98]]

 [[156 151 143]
  [156 150 144]
  [156 150 144]
  ...
  [ 92  97  96]
  [ 92  97  96]
  [ 94  99  98]]

 [[156 151 143]
  [156 150 144]
  [156 150 144]
  ...
  [ 91  96  95]
  [ 92  97  96]
  [ 93  98  97]]] in 14 length datastore
1/1 ━━━━━━━━

25-05-07 19:33:50 - find function duration 72.44706296920776 seconds25-05-07 19:33:50 - find function duration 70.26857018470764 seconds

25-05-07 19:33:50 - find function duration 67.44133710861206 seconds
25-05-07 19:33:51 - find function duration 75.08500862121582 seconds
